In [1]:
import os
import json
import re
import pandas as pd

In [2]:
# Regex de atos de aditamento

principal_aditivo = r'(?:ADITIVO)'
regex_titulo_aditivo = r'(?:(ADITIVO[S]*\s.*CONTRAT[OUALIS]*)|(CONTRATO[S]*\s.*ADITIVO[S]*)|(ADITIVO,\s.*CONTRATO))'
regex_texto_aditivo = r'(?:(aditivo\sao\scontrato)|(espécie:\scontrato)|(termo\saditivo\s-\sao\scontrato))'

titulos_termo_aditivo = [
    'EXTRATO DE TERMO ADITIVO',
    'EXTRATO DE ADITIVO',
    'EXTRATO DE TERMO ADITIVO (*)',
    'EXTRATOS DE TERMO ADITIVO',
    'EXTRATOS DE TERMOS ADITIVOS',
    'EXTRATO DO PRIMEIRO TERMO ADITIVO',
    'EXTRATO DE TERMO DE ADITIVO',
    'EXTRATO DE TERMOS ADITIVOS',
]

# Regex de atos de anulação e revogação

regex_anulacao_revogacao = r'(?:AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O)'

# Regex de atos de contrato e convênio

principal_contrato = r'(?:CONTRATO)'

regex_titulo_contrato = r'(?:(EXTRATO\sD[OE]\sCONTRATO)|(CONTRATO\sSIMPLIFICADO)|(CONTRATO\sPARA\sAQUISIÇÃO)|(^CONTRATO)'\
                        '|(EXTRATO\sD[OE]\sTERMO\sD[OE]\sCONTRATO\sNº\s)|(EXTRATO\sAO\sCONTRATO)|(CONTRATO\sDE\sPRESTAÇÃO\sDE\sSERVIÇOS\sNº)'\
                        '|(CONTRATO\sDE\sPATROCÍNIO)|(CONTRATO\sDE\sCONCESSÃO\sDE)|(^CONTRATO\sNº)|(^EXTRATOS\sDE\sCONTRATO[S]*))'

principal_convenio = r'(?:(CONVÊNIO)|(CONVENIO))'

regex_titulo_convenio = r'(?:(EXTRATO\sD[OE]\sCONVÊNIO)|(EXTRATO\sDE\sTERMO\sDE\sCONVÊNIO)|(CONVÊNIO\sSIMPLIFICADO)|(CONVÊNIO\sPARA\sAQUISIÇÃO)|(^CONVÊNIO)'\
                        '|(EXTRATO\sD[OE]\sTERMO\sD[OE]\sCONVÊNIO\sNº\s)|(EXTRATO\sAO\sCONVÊNIO)|(CONVÊNIO\sDE\sPRESTAÇÃO\sDE\sSERVIÇOS\sNº)'\
                        '|(CONVÊNIO\sDE\sPATROCÍNIO)|(CONVÊNIO\sDE\sCONCESSÃO\sDE)|(^CONVÊNIO\sNº)|(^EXTRATOS\sDE\sCONVÊNIO[S]*))'

# Regex de atos de licitação

regex_licitacao = r'(?:AVISO\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+PREG[AÃ]O\s+ELETR[OÔ]NICO|AVISOS\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISOS\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISOS\s+D[EO]\s+PREG[AÃ]O\s+ELETR[OÔ]NICO|AVISOS\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][OÕ]ES|AVISOS?\s+D[EO]\s+LICITA[CÇ][OÕ]ES)'

# Regex de atos e suspensão

regex_suspensao = r'(?:AVISO\s+D[EO]\s+SUSPENS[AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+SUSPENS[AÃ]O)'

In [3]:
atos_aditamento = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[],
    'orgao': [],
    'documento': []
}

atos_anulacao_revogacao = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[],
    'orgao': [],
    'documento': []
}

atos_contrato_convenio = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[],
    'orgao': [],
    'documento': []
}

atos_licitacao = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[],
    'orgao': [],
    'documento': []
}

atos_suspensao = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[],
    'orgao': [],
    'documento': []
}

outros = {
    'numero_dodf':[],
    'orgao': [],
    'documento': [],
    'titulo':[],
    'texto':[]
}

In [4]:
def capture_acts(path):
    with open(path, 'r') as f:
        file = json.load(f)
        try:
            section_3 = file['json']['INFO']['Seção III']
            for orgao in section_3:
                for documento in section_3[orgao]:
                    for ato in section_3[orgao][documento]:
                        titulo = section_3[orgao][documento][ato]['titulo']
                        texto = section_3[orgao][documento][ato]['texto']
                        
                        if re.search(principal_aditivo, titulo) is not None:
                            atos_aditamento['numero_dodf'].append(file['json']['nu_numero'])
                            atos_aditamento['titulo'].append(titulo)
                            atos_aditamento['texto'].append(re.sub(r'<[^>]*>', '', texto))
                            atos_aditamento['orgao'].append(orgao)
                            atos_aditamento['documento'].append(ato)

                        elif titulo in titulos_termo_aditivo:
                            if re.search(regex_texto_aditivo, texto.lower()) is not None:
                                atos_aditamento['numero_dodf'].append(file['json']['nu_numero'])
                                atos_aditamento['titulo'].append(titulo)
                                atos_aditamento['texto'].append(re.sub(r'<[^>]*>', '', texto))
                                atos_aditamento['orgao'].append(orgao)
                                atos_aditamento['documento'].append(ato)
                        
                        elif re.search(regex_anulacao_revogacao, titulo) is not None:
                            atos_anulacao_revogacao['numero_dodf'].append(file['json']['nu_numero'])
                            atos_anulacao_revogacao['titulo'].append(titulo)
                            atos_anulacao_revogacao['texto'].append(re.sub(r'<[^>]*>', '', texto))
                            atos_anulacao_revogacao['orgao'].append(orgao)
                            atos_anulacao_revogacao['documento'].append(ato)
                        
                        elif re.search(principal_contrato, titulo) is not None:
                            if re.search(r'(?:ADITIVO)', titulo) is None:
                                if re.search(regex_titulo_contrato, titulo) is not None:
                                    if 'termo aditivo ao contrato' not in texto.lower():
                                        atos_contrato_convenio['numero_dodf'].append(file['json']['nu_numero'])
                                        atos_contrato_convenio['titulo'].append(titulo)
                                        atos_contrato_convenio['texto'].append(re.sub(r'<[^>]*>', '', texto))
                                        atos_contrato_convenio['orgao'].append(orgao)
                                        atos_contrato_convenio['documento'].append(ato)
                        
                        elif re.search(principal_convenio, titulo) is not None:
                            if re.search(r'(?:ADITIVO)', titulo) is None:
                                if re.search(regex_titulo_convenio, titulo) is not None:
                                    if 'termo aditivo ao con' not in texto.lower():
                                        atos_contrato_convenio['numero_dodf'].append(file['json']['nu_numero'])
                                        atos_contrato_convenio['titulo'].append(titulo)
                                        atos_contrato_convenio['texto'].append(re.sub(r'<[^>]*>', '', texto))
                                        atos_contrato_convenio['orgao'].append(orgao)
                                        atos_contrato_convenio['documento'].append(ato)

                        elif re.search(regex_licitacao, titulo) is not None:
                            atos_licitacao['numero_dodf'].append(file['json']['nu_numero'])
                            atos_licitacao['titulo'].append(titulo)
                            atos_licitacao['texto'].append(re.sub(r'<[^>]*>', '', texto))
                            atos_licitacao['orgao'].append(orgao)
                            atos_licitacao['documento'].append(ato)

                        elif re.search(regex_suspensao, titulo) is not None:
                            atos_suspensao['numero_dodf'].append(file['json']['nu_numero'])
                            atos_suspensao['titulo'].append(titulo)
                            atos_suspensao['texto'].append(re.sub(r'<[^>]*>', '', texto))
                            atos_suspensao['orgao'].append(orgao)
                            atos_suspensao['documento'].append(ato)

                        else:
                            outros['numero_dodf'].append(file['json']['nu_numero'])
                            outros['orgao'].append(orgao)
                            outros['documento'].append(ato)
                            outros['titulo'].append(titulo)
                            outros['texto'].append(re.sub(r'<[^>]*>', '', texto))

        except KeyError:
            print(f"Chave 'Seção III' não encontrada no DODF {file['lstJornalDia']}!")


In [5]:
#pasta = './datasets/dodf_json_ate_25_08/'

pasta = './datasets/dodf_json_nov_22/' # jsons de novembro de 2022

#js = './datasets/DODF 002 04-01-2022.json'
#capture_acts(js)

In [6]:
dodfs = os.listdir(pasta)
for dodf in dodfs:
    if not dodf.endswith('.json'):
        continue

    path = pasta + dodf
    capture_acts(path)

In [7]:
df_atos_aditamento = pd.DataFrame(atos_aditamento)
df_atos_anulacao_revogacao = pd.DataFrame(atos_anulacao_revogacao)
df_atos_contrato_convenio = pd.DataFrame(atos_contrato_convenio)
df_atos_licitacao = pd.DataFrame(atos_licitacao)
df_atos_suspensao = pd.DataFrame(atos_suspensao)
df_outros = pd.DataFrame(outros)

In [8]:
df_atos_aditamento

,numero_dodf,titulo,texto,orgao,documento
0,212,EXTRATO DO SEGUNDO TERMO ADITIVO AO CONTRATO N...,PROCESSO SEI Nº 00054-00018174/2020-91 – PARTE...,Secretaria de Estado de Segurança Pública,258328
1,212,SEGUNDO TERMO ADITIVO AO TERMO DE CREDENCIAMEN...,"O Distrito Federal, por meio do Departamento d...",Secretaria de Estado de Segurança Pública,258176
2,212,PRIMEIRO TERMO ADITIVO AO TERMO DE CREDENCIAME...,"O Distrito Federal, por meio do Departamento d...",Secretaria de Estado de Segurança Pública,258145
3,212,PRIMEIRO TERMO ADITIVO AO TERMO DE CREDENCIAME...,"O Distrito Federal, por meio do Departamento d...",Secretaria de Estado de Segurança Pública,258146
4,212,EXTRATO DO 6º TERMO ADITIVO AO CONTRATO Nº 36/...,PROCESSO Nº: 00113-00005897/2021-32; CONTRATAN...,Secretaria de Estado de Transporte e Mobilidade,258290
5,212,EXTRATO DO SEGUNDO TERMO ADITIVO PARA REEQUILÍ...,PROCESSO Nº 00110-00000921/2022-11 (1º Aditivo...,Secretaria de Estado de Obras e Infraestrutura,258295
6,212,EXTRATO DE TERMO ADITIVO,"1º Termo Aditivo ao Contrato 9403/2021, public...",Secretaria de Estado de Obras e Infraestrutura,258274
7,212,EXTRATO DE TERMO ADITIVO,"1º Termo Aditivo ao Contrato 9438/2022, public...",Secretaria de Estado de Obras e Infraestrutura,258275
8,212,EXTRATO DE TERMO ADITIVO,"1º Termo Aditivo ao Contrato 9455/2022, public...",Secretaria de Estado de Obras e Infraestrutura,258273
9,212,TERMO ADITIVO À ATA DE REGISTRO DE PREÇOS,1° Termo Aditivo À Ata De Registro De Preços N...,Secretaria de Estado de Obras e Infraestrutura,258270


In [9]:
df_atos_anulacao_revogacao

,numero_dodf,titulo,texto,orgao,documento


In [17]:
df_atos_contrato_convenio['orgao'][5]

'Secretaria de Estado da Agricultura, Abastecimento e Desenvolvimento Rural'

In [11]:
df_atos_licitacao

,numero_dodf,titulo,texto,orgao,documento
0,212,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 147/2022 - UASG 974002A P...,"Secretaria de Estado de Planejamento, Orçament...",258258
1,212,AVISO DE LICITAÇÃO - ABERTURA,PREGÃO ELETRÔNICO Nº 74/2022 - DICOA/DEALF/CBM...,Secretaria de Estado de Segurança Pública,258225
2,212,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 116/2022A presente licita...,Secretaria de Estado de Transporte e Mobilidade,258288
3,212,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 117/2022A presente licita...,Secretaria de Estado de Transporte e Mobilidade,258289
4,212,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO PE Nº 259/2022-CAESBPROCESSO...,Secretaria de Estado de Obras e Infraestrutura,258265
5,212,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO PE Nº 260/2022-CAESBPROCESSO...,Secretaria de Estado de Obras e Infraestrutura,258266
6,212,AVISO DE ABERTURA DE LICITAÇÃO,O Presidente da Comissão de Licitação para Com...,Secretaria de Estado de Desenvolvimento Econômico,258307


In [12]:
df_atos_suspensao

,numero_dodf,titulo,texto,orgao,documento
0,212,AVISO DE SUSPENSÃO DO PREGÃO ELETRÔNICO Nº 16/...,PROCESSO SEI Nº 00220-00006337/2022-87(AMPLA C...,Secretaria de Estado de Esporte e Lazer,258230


In [19]:
df_outros['orgao'][8]

'Secretaria de Estado de Saúde'

# ANÁLISE JSON
- **DODF 212 11-11-2022**
    - 137 atos no total
    - 19 de aditamento
    - 7 de licitação
    - 7 de contrato/convênio
    - 1 de suspensão
    - 0 de anulação/revogação*

* Possíveis atos de aditamento não sendo classificados

Exemplo: documento nº 258437 da Secretaria de Estado de Saúde

**TITULO:** EXTRATO CONTRATUAL

**TEXTO:**

"Espécie: **Quinto Termo Adivito** ao Contrato nº 106/2017-SES/DF. SIGGO: 35515. Partes: DISTRITO FEDERAL, por intermédio de sua **SECRETARIA DE ESTADO DE SAÚDE DO DISTRITO FEDERAL** e a empresa **BRASÍLIA EMPRESA DE SEGURANÇA S/A**. CNPJ nº 02.730.521/0001-20. **Objeto**: prorrogação excepcional do prazo de vigência do Contrato, por até 12 (doze) meses, com início em 23/12/2022 e término em 22/12/2023, ou até a finalização do processo de nova contratação (00040-00028184/2021-66), com base no Art. 57, §4° da Lei n° 8.666/93 e alterações. Vigência: a contar da assinatura. Dotação Orçamentária: Unidade Orçamentária: 23901. Programa de Trabalho: 10122820285170052. Natureza da Despesa: 339037. Fonte de Recurso: 100000000. Nota de Empenho: 2022NE10854. Valor de empenho inicial: R$22.564,03 (vinte e dois mil quinhentos e sessenta e quatro reais e três centavos). Emitido em 21/10/2022. Sob o evento: 400091. Na modalidade: 3- Global. Despesa de Publicação: SES. Processo : 00060-00459703/2018-64. Data de Assinatura: 11/11/2022. Pela SES/DF: LUCILENE MARIA FLORÊNCIO DE QUEIROZ. Pela contratada: GLAUCO CARVALHO SOUZA. Testemunhas: ALANIA DE OLIVEIRA SILVA e FERNANDO URBANO NETO. Publicação do Ajuste Original: 11/01/2018."

-------------------

Também existem atos que possuem o mesmo título, mas com textos diferentes. É o caso dos documentos nº 258318 e nº 258319 da Secretaria de Estado de Desenvolvimento Econômico.
Ambos os títulos são EXTRATO DE INSTRUMENTO CONTRATUAL, porém, nos textos, um é do tipo "Termo de Cessão de Uso" e o outro do tipo "Primeiro Termo Aditivo ao Contrato".

*Além disso, também identifiquei um ato categorizado como contrato/convênio que ao que tudo indica, seria, na verdade, de anulação/revogação. (Documento nº 258429 da Secretaria de Estado da Agricultura, Abastecimento e Desenvolvimento Rural)

**TITULO:** EXTRATO DO TERMO DE **RESCISÃO** DO CONTRATO DE PRESTAÇÃO DE SERVIÇOS Nº 01/2020 – SEAGRI/DF

**TEXTO:**

PROCESSO SEI-GDF: 00070-00000132/2020-32. Partes: SEAGRI/DF e SECRETARIA DE AGRICULTURA E ABASTECIMENTO - INSTITUTO BIOLÓGICO. Objeto: O presente Termo objetiva a rescisão do Contrato de Prestação de Serviços nº 01/2020 – SEAGRI/DF, conforme determinado no Parágrafo Segundo, da Cláusula Segunda - Do Objeto, do Primeiro Termo Aditivo: "A prorrogação de vigência consignada no caput se dará por mais 12 (doze) meses, ou se ocorrer primeiro, na data de início da prestação dos serviços decorrentes da celebração do Contrato objeto do processo 00070-00001626/2021-15, conforme justificativa contida no Despacho Sei Id. 62743480", e com base no art. 79, II da Lei 8.666/93 rescindindo-se nesta data de pleno direito. Prazo de Vigência: O presente Termo entra em vigência a partir da data de sua assinatura. Data de Assinatura: 31/10/2022. Signatários: Pela SEAGRI/DF: CANDIDO TELES DE ARAÚJO na qualidade de Secretário de Estado, Pela Contratada: ANA EUGÊNIA DE CARVALHO CAMPOS, na qualidade de Diretora Técnica de Departamento.